In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install pmdarima

In [ ]:
import pandas as pd
import numpy as np
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.arima_model import ARIMA
from pmdarima import auto_arima
import matplotlib.pyplot as plt

In [ ]:
import os
from sklearn.preprocessing import StandardScaler
import itertools

# 경로 설정
base_path = '/content/drive/MyDrive/KHUDA 금융/심화트랙/데이터'
scores_path = os.path.join(base_path, '섹터별 일일 평균 점수 - 4차')
prices_path = os.path.join(base_path, '지수 종가')

# 섹터 리스트
sectors = [
    "건설", "경기소비재", "기계장비", "반도체", "방송통신", "보험",
    "에너지화학", "은행", "자동차", "정보기술", "증권", "철강", "헬스케어"
]

# 데이터를 저장할 리스트 초기화
sector_scores = {}
sector_prices = {}

# 섹터별 파일 반복 처리
for sector in sectors:
    try:
        # 파일 경로 생성
        score_file = os.path.join(scores_path, f'{sector}_sector_daily_weighted_scores.csv')
        price_file = os.path.join(prices_path, f'KRX {sector}.csv')

        # 데이터 로드
        sector_scores[sector] = pd.read_csv(score_file, encoding='utf-8', index_col=0)
        sector_prices[sector] = pd.read_csv(price_file, encoding='CP949', index_col=0)
        print(f"Loaded {sector} data successfully.")
    except Exception as e:
        print(f"Failed to load data for {sector}: {e}")

# 주가 데이터 컬럼 이름 변경 및 정렬
for sector in sectors:
    try:
        df = sector_prices[sector]
        df.rename(columns={'종가': 'Close'}, inplace=True)
        df.index.name = 'Date'
        df.index = pd.to_datetime(df.index)  # Index를 datetime 형식으로 변환
        df.sort_index(inplace=True)
        sector_prices[sector] = df
        print(f"Updated column names and sorted for sector: {sector}")
    except Exception as e:
        print(f"Failed to update data for sector: {sector}, Error: {e}")

# 1. 주가 데이터의 등락률 계산
def calculate_price_change(df_price, n_values):
    for n in n_values:
        df_price[f'Close_pct_change_{n}'] = (df_price['Close'] - df_price['Close'].shift(n)) / df_price['Close'].shift(n) * 100
    return df_price

# 2. 감성분석 데이터의 스케일링 및 이동평균 계산
def process_sentiment_data(df_score, moving_avg_windows):
    scaler = StandardScaler()
    df_score['Scaled_sentiment'] = scaler.fit_transform(df_score[['sector_weighted_score']])
    for window in moving_avg_windows:
        df_score[f'Sentiment_MA_{window}'] = df_score['Scaled_sentiment'].rolling(window=window).mean()
    return df_score

# 3. 데이터 병합 및 정리
def prepare_combined_data(df_price, df_score, n_values, moving_avg_windows):
    df_price = calculate_price_change(df_price, n_values)
    df_score = process_sentiment_data(df_score, moving_avg_windows)
    df_combined = pd.merge(df_price, df_score, left_index=True, right_index=True)  # Index 기준 병합
    df_combined.replace([np.inf, -np.inf], pd.NA, inplace=True)
    df_combined.dropna(inplace=True)
    return df_combined

# 4. ARIMAX 모델 실행 및 p-value 저장 (수정)
def run_arimax_models(df_combined, price_sector, score_sector, price_diffs, sentiment_windows):
    significant_results = []
    for price_diff, sentiment_window in itertools.product(price_diffs, sentiment_windows):
        try:
            # auto_arima로 최적 파라미터 탐색
            auto_arimax_model = auto_arima(
                df_combined[f'Close_pct_change_{price_diff}'],
                exogenous=df_combined[[f'Sentiment_MA_{sentiment_window}']],
                seasonal=False,
                trace=False,
                error_action='ignore',
                suppress_warnings=True
            )

            # SARIMAX 모델 적합
            sarimax_model = SARIMAX(
                df_combined[f'Close_pct_change_{price_diff}'],
                exog=df_combined[[f'Sentiment_MA_{sentiment_window}']],
                order=auto_arimax_model.order,
                enforce_stationarity=False,
                enforce_invertibility=False
            )
            sarimax_results = sarimax_model.fit(disp=False)

            # 외생변수 p-value 추출
            p_value = sarimax_results.pvalues[f'Sentiment_MA_{sentiment_window}']

            # p-value가 0.2 이하인 경우 저장
            if p_value <= 0.2:
                significant_results.append({
                    'price_sector': price_sector,
                    'score_sector': score_sector,
                    'price_diff': price_diff,
                    'sentiment_ma': sentiment_window,
                    'p_value': p_value
                })
        except Exception as e:
            print(f"Error during ARIMAX fitting: Price Diff={price_diff}, Sentiment MA={sentiment_window}, Error: {e}")
    return significant_results

# 5. 메인 실행 코드 (수정)
def main(sector_prices, sector_scores, sectors):
    # 설정값 정의
    price_diffs = [1, 5, 10, 15]
    sentiment_windows = [5, 10, 15]
    all_significant_results = []

    # 주가 섹터와 감성분석 섹터의 모든 조합 생성
    for price_sector, score_sector in itertools.product(sectors, sectors):
        try:
            print(f"\nStarting analysis for Price Sector: {price_sector}, Score Sector: {score_sector}")

            # 주가 섹터 데이터 로드
            df_price = sector_prices[price_sector]

            # 감성분석 섹터 데이터 로드
            df_score = sector_scores[score_sector]
            df_score.index = pd.to_datetime(df_score.index)  # 감성분석 데이터 Index를 datetime 형식으로 변환

            # 데이터 병합
            df_combined = prepare_combined_data(df_price, df_score, price_diffs, sentiment_windows)

            # ARIMAX 모델 실행
            significant_results = run_arimax_models(
                df_combined, price_sector, score_sector, price_diffs, sentiment_windows
            )
            all_significant_results.extend(significant_results)
        except Exception as e:
            print(f"Error processing Price Sector: {price_sector}, Score Sector: {score_sector}, Error: {e}")

    # 결과 저장
    if all_significant_results:
        output_df = pd.DataFrame(all_significant_results)
        output_path = os.path.join(base_path, 'significant_results.csv')
        output_df.to_csv(output_path, index=False, encoding='utf-8-sig')
        print(f"Significant results saved to: {output_path}")
    else:
        print("No significant results found.")

# 실행
main(sector_prices, sector_scores, sectors)



Loaded 건설 data successfully.
Loaded 경기소비재 data successfully.
Loaded 기계장비 data successfully.
Loaded 반도체 data successfully.
Loaded 방송통신 data successfully.
Loaded 보험 data successfully.
Loaded 에너지화학 data successfully.
Loaded 은행 data successfully.
Loaded 자동차 data successfully.
Loaded 정보기술 data successfully.
Loaded 증권 data successfully.
Loaded 철강 data successfully.
Loaded 헬스케어 data successfully.
Updated column names and sorted for sector: 건설
Updated column names and sorted for sector: 경기소비재
Updated column names and sorted for sector: 기계장비
Updated column names and sorted for sector: 반도체
Updated column names and sorted for sector: 방송통신
Updated column names and sorted for sector: 보험
Updated column names and sorted for sector: 에너지화학
Updated column names and sorted for sector: 은행
Updated column names and sorted for sector: 자동차
Updated column names and sorted for sector: 정보기술
Updated column names and sorted for sector: 증권
Updated column names and sorted for sector: 철강
Updated column names and sort

/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)



Starting analysis for Price Sector: 건설, Score Sector: 경기소비재


/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)



Starting analysis for Price Sector: 건설, Score Sector: 기계장비


...


Starting analysis for Price Sector: 헬스케어, Score Sector: 철강


/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)



Starting analysis for Price Sector: 헬스케어, Score Sector: 헬스케어


/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


Significant results saved to: /content/drive/MyDrive/KHUDA 금융/심화트랙/데이터/significant_results.csv
